###### https://www.youtube.com/watch?v=i_LwzRVP7bg&list=PLWKjhJtqVAblStefaz_YOVpDWqcRScc2s&index=1
###### https://colab.research.google.com/drive/1m3oQ9b0oYOT-DXEy0JCdgWPLGllHMb4V?usp=sharing#scrollTo=17bmL-NpKUhE
###### https://colab.research.google.com/drive/16w3TDn_tAku17mum98EWTmjaLHAJcsk0?usp=sharing#scrollTo=-Vohv6aAzZFK

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# from sklearn.preprocessing import StandardScaler
# from imblearn.over_sampling import RandomOverSampler

## data import

In [ ]:
cols = ["fLength", "fWidth", "fSize", "fConc", "fConc1", "fAsym", "fM3Long", "fM3Trans", "fAlpha", "fDist", "class"]
df=pd.read_csv("magic04.data",names=cols)


In [8]:
df.head()

,fLength,fWidth,fSize,fConc,fConc1,fAsym,fM3Long,fM3Trans,fAlpha,fDist,class
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,1
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,1
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,1
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,1
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,1


In [ ]:
df["class"] = (df["class"] == "g").astype(int)

In [ ]:
for label in cols[:-1]:
  plt.hist(df[df["class"]==1][label], color='blue', label='gamma', alpha=0.7, density=True)
  plt.hist(df[df["class"]==0][label], color='red', label='hadron', alpha=0.7, density=True)
  plt.title(label)
  plt.ylabel("Probability")
  plt.xlabel(label)
  plt.legend()
  plt.show()

### daily fluctuation analyse 

In [ ]:
df['date'] = pd.to_datetime(df['date'])

df['candle_fluctuation'] = df['high'] - df['close']
# daily_fluctuation = df.set_index('timestamp').groupby(pd.Grouper(freq='D'))['candle_fluctuation'].max()
df['daily_fluctuation'] = df.groupby(df['date'].dt.date)['candle_fluctuation'].transform('max')
df['previous_daily_fluctuation'] = df['daily_fluctuation'].shift(24)

df.dropna(subset=['previous_daily_fluctuation'], 
        inplace=True)

### weekday fluctuation hist 

In [ ]:
df['weekday_number'] = df['date'].dt.weekday
# df['range'] = df['high'] - df['low']
df['price_move'] = df['close'] - df['open']
df["class"] = (df["close"]>df["open"]).astype(int)

In [ ]:
def show_hist(df, wn=0):
    day_name_map = {0:"Monday", 1:"Tuesday", 2:"Wednesday", 3:"Thursday", 4:"Friday", 5:"Saturday", 6:"Sunday"}
    conditions = [
    df['weekday_number']==wn,
    df['price_move']<600,
    df['price_move']>-600,
    ]
    plt.hist(
        df[reduce(lambda x, y: x & y, conditions)]['price_move'], 
        color='g', alpha=0.7, density=True)
    plt.title(day_name_map[wn])
    plt.show